# MODEL
    ! pip install -U git+https://github.com/huggingface/transformers.git
    ! pip install -U git+https://github.com/huggingface/accelerate.git

In [ ]:
#AutoModel只是模型的主干部分，不包含头，不是模型的整体
from transformers import AutoModel

#分成两种模式
#model =AutoModel.from_config(config)#传入上面实例化的参数，要看cofig文件里面、
model = AutolModel.from_pretrained('albert-xlarge-v1')#--》AlbertModel ,eval()模式
#   ModelConfig ---->>> Model

In [17]:
#AutoModelForXXX  才是整体的模型，一个模型可能会服务多个任务，都有独自的类，区别是损失的计算
#自动映射到对应的模型
from transformers import AutoModelForPreTraining,AutoModelForSequenceClassification
#分两种模式
#model  = AutoModelForPreTraining.from_config(config)
#model =AutoModelForSequenceClassification.from_config(config)
model = transformers.AutoModelForImageClassification.from_pretrained('microsoft/resnet-50',config=config,ignore_mismatched_sizes=True)

In [ ]:
# 在模型文件中，模型的特定大小由ALBERT_PRETRAINED_MODEL_ARCHIVE_LIST 来决定

# 自定义模型


In [2]:
from transformers.modeling_utils import PreTrainedModel
from transformers.utils import ModelOutput
from dataclasses import dataclass

In [ ]:

#实现模型的所有组件模块，直接继承nn.Module即可

#制作该模型的输出类
#每个模块都配有一个输出类，都是继承ModelOutput，看模型分几个部分，自定义
@dataclass
class ModuleForPreTrainingOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    prediction_logits: torch.FloatTensor = None
    sop_logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None
        
#PreTrainedModel 中做了大量的实例化工作，包括量化
class XmodPreTrainedModel(PreTrainedModel)
    #首先写一个父类，用来初始化模型
    def _init_weights(self, module):
        """Initialize the weights."""
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)   

#这个是模型主干
class XmodModel(XmodPreTrainedModel):

    #再写一个组建函数，把所有的模块组建在一起,就是模型的forward
    def __init__(self, config: AlbertConfig):
        super().__init__(config)
        
        self.config = config
        
        self.module = module(config)
        
        self.post_init()
        
    def forward(self,
        pixel_values: torch.FloatTensor = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,#可以选择输出是什么类型
):
        output = calc(**inputs)
        
        if not return_dict:
            return (sequence_output, pooled_output) + encoder_outputs[1:]
        
        return BaseModuleOutput(output)#这个也要实现输出类

#这个是具体任务的模型,需要计算损失和解码的部分，包含头
class XmodForSequenceClassification(XmodPreTrainedModel)
    #最后的整合各个组建的类
    #forward要计算出loss，再对结果进行包装,取出loss可以使用 output['loss']
    def __init__(self, config: AlbertConfig):
        super().__init__(config)
        
        self.backbone=Backbone()
        
        self.model = XmodModel()
        self.head = Head()
        self.post_init()
    def forward(self,
        pixel_values: torch.FloatTensor = None,
        labels: Optional[torch.LongTensor] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.model(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        
        #计算出损失
        total_loss=None
        if labels is not None:
            total_loss= self.head(outputs,labels)
        
        if not return_dict:
            output = (prediction_scores, sop_scores) + outputs[2:]
            return ((total_loss,) + output) if total_loss is not None else output

        return AlbertForPreTrainingOutput(
            loss=total_loss,#如果没有给定标签，就是None
            prediction_logits=prediction_scores,#预测的结果
            sop_logits=sop_scores,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


In [ ]:
PeftAdapterMixin：
     PEFT library调参库，对大语言模型，只要微调小部分参数即可

PushToHubMixin：
    push a model/tokenizer to the hub
    push_to_hub函数
    
GenerationMixin：
    auto-regressive text generation
    generate函数
    
ModuleUtilsMixin:
    nn.Module的补充部分

所有的Auto模块都是完成“字符串到类的映射”，之后调用映射类中的from_pretrained完成实例化

_BaseAutoModelClass：所有模块的基类
    _model_mapping：字典映射，把字符串映射为对应的模块

    只有3个方法，强调所有继承这个的子模型，都只能用这三个方法
    from_config：
    from_pretrained：
    register：

In [ ]:

            AlbertModel

            AlbertForPreTraining

            AlbertForMaskedLM
    
albert----》AlbertForSequenceClassification
        
            AlbertForQuestionAnswering
            
            AlbertForTokenClassification
            
            AlbertForMultipleChoice

In [ ]:
Trainer 的方法让模型必须实现outputs = model(**inputs)返回损失字典